# Recurrent Neural Networks 

The models previously explored predict the value of $x_t$ at some time $t$ based on the previous $n-1$ observed values. However, if we wish to increase the history which is used to predict the next value, we must increase $n$, which results in a $\mathbb{V}^n$ increase in required model parameters for a vocabulary of size $\mathbb{V}$. 

Instead, it makes sense to use a "latent variable" model, where the prediction at each timestep t is instead dependent on a hidden state, which stores the sequence information up to step t-1. 

$$ P(x_t | x_{t - 1}, x_{t - 2} ... x_{1}) = P(x_t | h_{t-1}) $$, 

Where $h_{t-1}$ is the hidden state which stores sequence information up to $t-1$. In general, the hidden state at any time step $t$ can be compuyted based on the current input and the value of the previous hidden state. 

It is important to draw a distinction between a *hidden layer* and a *hidden state*. A hidden layer is simply a layer that is hidden on the path from input to output. A hidden *state* is technically speaking an input to the model, which can only be computed using the data from the previous timesteps. 

In [1]:
import torch
from d2l import torch as d2l

## Neural Networks Without Hidden States

Consider a single MLP with a hidden layer, let the hidden layer's activation be $\phi$. Given a minibatch of examples $\mathbf{X} \in \mathbb{R}^{n \times d}$ with batch size $n$ and $d$ inputs, the hidden layer output is computed as 

$$ \mathbf{H} = \phi (\mathbf{XW_{xh}} + b_h) $$. 

Where we have the weight parameter $\mathbf{W}_{xh} in \mathbb{R}^{d \times h}$ and the bias parameter $\mathbf{b} \in \mathbb{R}^{1 \times h}$. Where h is the numebr of hidden units. The output of the MLP is computer analogously from the weights of the hidden layer. 

## Recurrent Neural Networks with a Hidden State

The situation is rather different for a neural network with a hidden state. As before, we have a minibatch of inputs $X \in \mathbb{R}^{n \times d}$, which contains $n$ examples where each row ($d$) contains the information for that example at time step $t$. Next, we can denote by $\mathbf{H}_t \in \mathbb{R} ^ {n \times h}$ the matrix of weights in the hidden layer for timestep t. Unlike in the previous example, here we _save_ the previous hidden layer, $\mathbf{H}_{t-1}$, and introduce a new weight parameter $\mathbf{W}_{hh}$, which specifies how the weights of the previous hidden layer are to be used in the computation of the new output. Finally, the weights of the hidden layer at time $t$ are computed with

$$ H_t = \phi (X_tW_{xh} + H_{t-1}W_{hh} + b_h) $$.

The additional middle term creates the relationship between each step and the previous. Because these hidden states are defined and used repeatedly, they are recurrent. Layers which implement this computation are known as recurrent layers. 

The matrices of weights are only dependent on the size of the hidden layers and the length of the data in each example, so the cost of evaluating a recurrent neural network does not increase with increasing sequence length. 

In practise, the expression involving the separate multiplication and addition of two sets of weights and matrices is achieved by concatenating the matrices $X_t$ and $H_{t-1}$ and the matrices $W_{xh}$ and $W_{hh}$ and taking the matrix product of these two. This is what is done in practise, thereby performing only one matrix multiplication operation. 

In [2]:
X, W_xh = torch.randn(3, 1), torch.randn(1, 4)
H, W_hh = torch.randn(3, 4), torch.randn(4, 4)
torch.matmul(X, W_xh) + torch.matmul(H, W_hh)

tensor([[ 2.6667, -0.3103, -0.7596,  1.8122],
        [-0.1391, -0.5460, -0.8374,  2.0272],
        [ 2.3702,  2.4148,  1.1735, -0.8247]])

In [5]:
# Concatenate X and H along the columns and the weights along the rows....
torch.matmul(torch.cat((X, H), 1), torch.cat((W_xh, W_hh), 0))

tensor([[ 2.6667, -0.3103, -0.7596,  1.8122],
        [-0.1391, -0.5460, -0.8374,  2.0272],
        [ 2.3702,  2.4148,  1.1735, -0.8247]])